In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession. \
            builder. \
            master('local'). \
            appName("practice"). \
            getOrCreate()

In [5]:
spark

In [6]:
order_df = spark.read.json(r"C:\Users\Kshitij kadu\Desktop\retail\orders")

In [7]:
order_df.count()

68883

In [8]:
customer_df = spark.read.json(r"C:\Users\Kshitij kadu\Desktop\retail\customers")

In [9]:
customer_df.count()

12435

In [10]:
from pyspark.sql.functions import *

In [15]:
df_join = order_df.join(broadcast(customer_df),order_df.order_customer_id == customer_df.customer_id)

In [16]:
df_join.explain()

== Physical Plan ==
*(2) BroadcastHashJoin [order_customer_id#7L], [customer_id#34L], Inner, BuildRight, false
:- *(2) Filter isnotnull(order_customer_id#7L)
:  +- FileScan json [order_customer_id#7L,order_date#8,order_id#9L,order_status#10] Batched: false, DataFilters: [isnotnull(order_customer_id#7L)], Format: JSON, Location: InMemoryFileIndex[file:/C:/Users/Kshitij kadu/Desktop/retail/orders], PartitionFilters: [], PushedFilters: [IsNotNull(order_customer_id)], ReadSchema: struct<order_customer_id:bigint,order_date:string,order_id:bigint,order_status:string>
+- BroadcastExchange HashedRelationBroadcastMode(List(input[3, bigint, false]),false), [id=#146]
   +- *(1) Filter isnotnull(customer_id#34L)
      +- FileScan json [customer_city#31,customer_email#32,customer_fname#33,customer_id#34L,customer_lname#35,customer_password#36,customer_state#37,customer_street#38,customer_zipcode#39] Batched: false, DataFilters: [isnotnull(customer_id#34L)], Format: JSON, Location: InMemoryFileIndex

repartitioning and coalesce

In [ ]:
repartitioning - It is use to increase the partitions

In [ ]:
coalesce - it is use to decrease the partitions

In [17]:
help(df_join.write.json)

Help on method json in module pyspark.sql.readwriter:

json(path, mode=None, compression=None, dateFormat=None, timestampFormat=None, lineSep=None, encoding=None, ignoreNullFields=None) method of pyspark.sql.readwriter.DataFrameWriter instance
    Saves the content of the :class:`DataFrame` in JSON format
    (`JSON Lines text format or newline-delimited JSON <http://jsonlines.org/>`_) at the
    specified path.
    
    .. versionadded:: 1.4.0
    
    Parameters
    ----------
    path : str
        the path in any Hadoop supported file system
    mode : str, optional
        specifies the behavior of the save operation when data already exists.
    
        * ``append``: Append contents of this :class:`DataFrame` to existing data.
        * ``overwrite``: Overwrite existing data.
        * ``ignore``: Silently ignore this operation if data already exists.
        * ``error`` or ``errorifexists`` (default case): Throw an exception if data already                 exists.
    compressi

In [24]:
df_join.repartition(4).write.json(r"C:\Users\Kshitij kadu\Desktop\retail\write\temp_repartition", mode = 'overwrite')

If we try to increase the amount of partiton using coalesce - It will not increase the
   partition and It will not throw an exception 

In [25]:
df_join.coalesce(4).write.json(r"C:\Users\Kshitij kadu\Desktop\retail\write\temp_coalesce", mode = 'overwrite')

In [ ]:
Spark transformations - 

Spark Transformation is a function that produces new RDD from
the existing RDDs. It takes RDD as input and produces one or more RDD as output. 
Each time it creates new RDD when we apply any transformation

transformation is an lazy evaluation


In [42]:
emp_df = spark.read.csv(r'C:\Users\Kshitij kadu\Desktop\sample_data\Records.csv', header = True, inferSchema = True)

In [43]:
emp_df.printSchema()

root
 |-- Emp ID: integer (nullable = true)
 |-- Name Prefix: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Middle Initial: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- E Mail: string (nullable = true)
 |-- Father's Name: string (nullable = true)
 |-- Mother's Name: string (nullable = true)
 |-- Mother's Maiden Name: string (nullable = true)
 |-- Date of Birth: string (nullable = true)
 |-- Time of Birth: string (nullable = true)
 |-- Age in Yrs.: double (nullable = true)
 |-- Weight in Kgs.: integer (nullable = true)
 |-- Date of Joining: string (nullable = true)
 |-- Quarter of Joining: string (nullable = true)
 |-- Half of Joining: string (nullable = true)
 |-- Year of Joining: integer (nullable = true)
 |-- Month of Joining: integer (nullable = true)
 |-- Month Name of Joining: string (nullable = true)
 |-- Short Month: string (nullable = true)
 |-- Day of Joining: integer (nullable = true)
 |

In [45]:
emp_df = emp_df.select("Emp id","First Name",\
               "last Name", "Gender", "E Mail","Date of Joining","Salary","County","City", "State")

In [46]:
emp_df.show()

+------+----------+---------+------+--------------------+---------------+------+------------+---------------+-----+
|Emp id|First Name|last Name|Gender|              E Mail|Date of Joining|Salary|      County|           City|State|
+------+----------+---------+------+--------------------+---------------+------+------------+---------------+-----+
|198429|  Serafina|Bumgarner|     F|serafina.bumgarne...|       2/1/2008| 69294|  Chautauqua|         Clymer|   NY|
|178566|  Juliette|     Rojo|     F|juliette.rojo@yah...|       6/4/2011|193912|  Montgomery|       Glenside|   PA|
|647173|     Milan| Krawczyk|     M|milan.krawczyk@ho...|      1/19/2012|123681|Anne Arundel|  Gibson Island|   MD|
|847634|     Elmer|    Jason|     M|elmer.jason@yahoo...|      5/28/2017| 93504|  Washington|        Mendota|   VA|
|260736|     Zelda|   Forest|     F|zelda.forest@ibm.com|      1/28/2014|176642| Schenectady|    Schenectady|   NY|
|811306|     Rhett|      Wan|     M|rhett.wan@hotmail...|      1/21/2009

In [59]:
emp_df.filter("Gender = 'M' and Salary > 20000").show()

+------+----------+-----------+------+--------------------+---------------+------+---------------+-------------+-----+
|Emp id|First Name|  last Name|Gender|              E Mail|Date of Joining|Salary|         County|         City|State|
+------+----------+-----------+------+--------------------+---------------+------+---------------+-------------+-----+
|647173|     Milan|   Krawczyk|     M|milan.krawczyk@ho...|      1/19/2012|123681|   Anne Arundel|Gibson Island|   MD|
|847634|     Elmer|      Jason|     M|elmer.jason@yahoo...|      5/28/2017| 93504|     Washington|      Mendota|   VA|
|811306|     Rhett|        Wan|     M|rhett.wan@hotmail...|      1/21/2009| 59406|         Fresno|        Selma|   CA|
|956633|       Hal|     Farrow|     M|  hal.farrow@cox.net|      2/25/1991|164580|     Stanislaus|      Modesto|   CA|
|629539|       Del|  Fernandez|     M|del.fernandez@hot...|       4/7/2016|138662|        Portage|         Kent|   OH|
|784160|     Corey|    Jackman|     M|corey.jack

In [53]:
emp_df.groupBy("State").count().show()

+-----+-----+
|State|count|
+-----+-----+
|   SC|  132|
|   AZ|  145|
|   LA|  168|
|   MN|  253|
|   NJ|  189|
|   DC|   66|
|   OR|   99|
|   VA|  303|
|   RI|   22|
|   KY|  260|
|   WY|   41|
|   NH|   59|
|   MI|  292|
|   NV|   49|
|   WI|  218|
|   ID|   84|
|   CA|  618|
|   CT|  111|
|   NE|  143|
|   MT|   99|
+-----+-----+
only showing top 20 rows



In [61]:
emp_df.cache()

DataFrame[Emp id: int, First Name: string, last Name: string, Gender: string, E Mail: string, Date of Joining: string, Salary: int, County: string, City: string, State: string]

In [ ]:
Broadcast join

In [62]:
df_join = order_df.join(broadcast(customer_df),order_df.order_customer_id == customer_df.customer_id).cache()

In [63]:
df_join.show()

+-----------------+--------------------+--------+---------------+-------------+--------------+--------------+-----------+--------------+-----------------+--------------+--------------------+----------------+
|order_customer_id|          order_date|order_id|   order_status|customer_city|customer_email|customer_fname|customer_id|customer_lname|customer_password|customer_state|     customer_street|customer_zipcode|
+-----------------+--------------------+--------+---------------+-------------+--------------+--------------+-----------+--------------+-----------------+--------------+--------------------+----------------+
|            11599|2013-07-25 00:00:...|       1|         CLOSED|      Hickory|     XXXXXXXXX|          Mary|      11599|        Malone|        XXXXXXXXX|            NC|8708 Indian Horse...|           28601|
|              256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|      Chicago|     XXXXXXXXX|         David|        256|     Rodriguez|        XXXXXXXXX|            IL

In [ ]:
StorageLevel.DISK_ONLY

In [65]:
from pyspark import StorageLevel

In [66]:
df_join = order_df.join(broadcast(customer_df),order_df.order_customer_id == customer_df.customer_id).persist(StorageLevel.DISK_ONLY)

In [67]:
df_join.show()

+-----------------+--------------------+--------+---------------+-------------+--------------+--------------+-----------+--------------+-----------------+--------------+--------------------+----------------+
|order_customer_id|          order_date|order_id|   order_status|customer_city|customer_email|customer_fname|customer_id|customer_lname|customer_password|customer_state|     customer_street|customer_zipcode|
+-----------------+--------------------+--------+---------------+-------------+--------------+--------------+-----------+--------------+-----------------+--------------+--------------------+----------------+
|            11599|2013-07-25 00:00:...|       1|         CLOSED|      Hickory|     XXXXXXXXX|          Mary|      11599|        Malone|        XXXXXXXXX|            NC|8708 Indian Horse...|           28601|
|              256|2013-07-25 00:00:...|       2|PENDING_PAYMENT|      Chicago|     XXXXXXXXX|         David|        256|     Rodriguez|        XXXXXXXXX|            IL